This code is to appear in a metaheuristic optimization book that is currently being written by Bogumil Kaminski, Alaa Khamis, Pawel Pralat and Przemyslaw Szufel 

In [1]:
println("Working directory : $(pwd())")

Working directory : C:\ComplexNetworks2019\day4


In [2]:
using Pkg
Pkg.activate(".")

"C:\\ComplexNetworks2019\\day4\\Project.toml"

In [3]:
using Random, DelimitedFiles

Load reference data 

In [4]:
const DIST = readdlm("distkm.txt")

153×153 Array{Float64,2}:
   0.0      25.5691   81.8831  388.798   …  366.945    32.1738  147.782 
  25.1537    0.0     102.777   413.394      391.541    56.0079  172.378 
  82.3007  103.224     0.0     399.893      378.04     50.7209  158.877 
 388.747   413.54    399.865     0.0         36.9392  367.115   241.551 
  96.4183  117.187   129.688   303.684      281.831    82.3623   62.6679
  31.5325   50.8019   52.7316  406.836   …  384.982    39.8438  165.82  
 359.209   334.3     401.386   745.918      724.065   381.755   504.902 
  83.9051  104.139    29.3792  428.76       406.907    65.2319  187.744 
   ⋮                                     ⋱    ⋮                         
 116.377   141.17    112.958   295.812      272.824    94.7451   56.111 
 143.864   163.623   178.233   366.067      344.213   129.714   136.748 
  34.8408   10.1703  103.898   421.578      399.724    59.6691  180.562 
 370.41    395.203   381.528    26.0345      12.0937  348.778   223.214 
 366.305   391.098   377.

Tour length calculation function common for all algorithms

In [5]:
function eval_tour(tour, dist)
    travel = dist[tour[end], tour[1]]
    for i in 2:length(tour)
        travel += dist[tour[i-1], tour[i]]
    end
    travel
end

eval_tour (generic function with 1 method)

Tabu search routine

In [8]:
function tabusearch(tabulength, iter, initial_s,
                    list_neighbours::Function, eval_s::Function)
    t = initial_s
    T = typeof(t)
    best_t, best_v = t, eval_s(t)
    tabulist = Vector{Union{T, Nothing}}(nothing,tabulength)
    tabupos = 1
    tabulist[tabupos] = t
    for i in 1:iter
        neighbours_evals = [(eval_s(nei), nei) for nei in list_neighbours(t)]
        sort!(neighbours_evals)
        found = false
        for (neiv, nei) in neighbours_evals
            if !(nei in tabulist)
                found = true
                tabupos = mod1(tabupos + 1, tabulength)
                tabulist[tabupos] = nei
                if neiv < best_v
                    best_t, best_v = nei, neiv
                end
                t = nei
                break
            end
        end
        if !found
            break
        end
    end
    (best_t, best_v)
end

tabusearch (generic function with 1 method)

In [9]:
function list_neighbours_2opt(tour)
    neighbours = Vector{typeof(tour)}()
    n = length(tour)
    for x in 1:n-1, y in x+1:n
        neighbour = copy(tour)
        reverse!(@view neighbour[x:y])
        push!(neighbours, neighbour)
    end
    neighbours
end

list_neighbours_2opt (generic function with 1 method)

In [12]:
init_tour(n) = randperm(n)
routea = tabusearch(1, 10^3, init_tour(size(DIST,1)),list_neighbours_2opt, (tour)->eval_tour(tour,DIST)  )

([139, 28, 99, 100, 103, 149, 88, 87, 12, 89  …  30, 50, 7, 48, 25, 117, 21, 53, 76, 10], 7076.809149620698)

In [13]:
routea

([139, 28, 99, 100, 103, 149, 88, 87, 12, 89  …  30, 50, 7, 48, 25, 117, 21, 53, 76, 10], 7076.809149620698)

In [14]:
function load_pos(file="walmart_node_locations.txt")
    pos = Dict{Int,Tuple{Float64,Float64}}()
    for line in readlines(file)
        els = split(line,"\t")
        pos[parse(Int,els[1])] = (parse(Float64,els[2]),parse(Float64,els[3]))
    end
    pos
end
function load_routes(file="walmart_routes.txt")
    routes = Matrix{Vector{Int}}(undef,size(DIST)...)
    fill!(routes,Int[])
    open(file) do f
        while !eof(f)
            line = split(readline(f),"\t")
            i,j = parse.(Int,line[2:3])
            routes[i,j]= parse.(Int,split(line[12],"#"))
        end
    end
    routes
end

pos, routes = (load_pos(), load_routes());

In [15]:
#required installation for map vizualisation
#using Conda
#Conda.runconda(`install folium -c conda-forge`)
using PyCall
flm = pyimport("folium")

function plot_tour_folium(t::Vector{Int}, dist::Array{Float64}, pos, routes)
    
    k = first(keys(pos))
    MAP_BOUNDS = [[pos[k]...],[pos[k]...]]
    for v in values(pos)
        (v[1] < MAP_BOUNDS[1][1]) && (MAP_BOUNDS[1][1]=v[1])
        (v[2] < MAP_BOUNDS[1][2]) && (MAP_BOUNDS[1][2]=v[2])
        (v[1] > MAP_BOUNDS[2][1]) && (MAP_BOUNDS[2][1]=v[1])
        (v[2] > MAP_BOUNDS[2][2]) && (MAP_BOUNDS[2][2]=v[2])
    end
    
    m = flm.Map()
    for k=1:length(t)
        i = t[k]
        j = t[k<length(t) ? (k+1) : 1]
        info = "Route $(k)-th <br>from=$i to=$j\n<br>distance=$(round(dist[i,j],digits=3))"
        flm.PolyLine(
            [pos[n] for n in routes[i,j]],
            popup=info,
            tooltip=info
        ).add_to(m)
        
    end
       

    for k=1:length(t)
        i = t[k]
        j = t[k<length(t) ? (k+1) : 1]
        info = "Node $(k)-th <br>number=$i next=$j\n<br>\n$(round.(pos[routes[i,j][1]],digits=6))"
        flm.Circle(
          location=pos[routes[i,j][1]],
          popup=info,
          tooltip=info,
          radius=2000,
          color="crimson",
          fill=true,
          fill_color="crimson"
       ).add_to(m)
    end
    
    m.fit_bounds(Tuple.(MAP_BOUNDS))
    m
end


plot_tour_folium (generic function with 1 method)

In [18]:
plot_tour_folium(routea[1], DIST, pos, routes)

PyObject <folium.folium.Map object at 0x000000003FC34860>